"""
@author: David Herron
"""

---

### This notebook explores the use of LogMap, a Java application for ontology alignment, from Python, via the Python package JPype.

The functionality exercised here demonstrates key aspects of the technical solution to be adopted for the product LogMap-LLM, a Python package that will wrap LogMap and integrate access to LLM models acting as Oracles to help decide whether and how entities from two ontologies might be related.

---

Basic imports

In [1]:
import os.path
import tomllib

JPype imports

In [2]:
# Import the module
import jpype

# Allow Java modules to be imported
import jpype.imports

# Import all standard Java types into the global scope
from jpype.types import *

Load the LogMap-LLM configuration file

In [3]:
with open("logmapllm.toml", mode="rb") as fp:
    config = tomllib.load(fp)

Show some of the configuration parameters

In [4]:
print(f'onto source: {config['alignmentTask']['onto_source']}')
print(f'onto target: {config['alignmentTask']['onto_target']}')
print()
print(f'oracle prompt template: {config['oraclePrompting']['oracle_prompt_template']}')
print()
print(f'openrouter apikey: {config['oracle']['openrouter_apikey']}')
print(f'openrouter LLM model name: {config['oracle']['openrouter_model_name']}')
print()
print(f'consult oracle: {config['pipeline']['consult_oracle']}')
print()
print(f'outputs dir: {config['outputs']['outputs_dir']}')

onto source: /Users/dave/research/logmap-usage/ontologies/mouse.owl
onto target: /Users/dave/research/logmap-usage/ontologies/human.owl

oracle prompt template: oracle prompt template name

openrouter apikey: openrouter apikey
openrouter LLM model name: openrouter LLM model name

consult oracle: True

outputs dir: path/to/outputs/directory/


Build JVM classpath and JVM options

In [5]:
logmap_dir = '/Users/dave/research/logmap-matcher-standalone-nov-2025/'

# path to main LogMap jar file
logmap_jar = os.path.join(logmap_dir, 'logmap-matcher-4.0.jar')
jpype.addClassPath(logmap_jar)

# path to LogMap dependency jar files
logmap_dep = os.path.join(logmap_dir, 'java-dependencies/*')
jpype.addClassPath(logmap_dep)

# LogMap jvm options
jvmOptions = [
    "-Xms500M", 
    "-Xmx25G",
    "-DentityExpansionLimit=10000000",
    "--add-opens=java.base/java.lang=ALL-UNNAMED"
]

Check if a JVM (Java Virtual Machine) is running

In [6]:
if jpype.isJVMStarted():
    print("JVM running, version:", jpype.getJVMVersion())
else:
    print("JVM is not running!")

JVM is not running!


Start a JVM

In [7]:
if not jpype.isJVMStarted():
    jpype.startJVM(*jvmOptions)

Confirm a JVM is running

In [8]:
if not jpype.isJVMStarted():
    print("JVM is not running!")
else:
    print("JVM version:", jpype.getJVMVersion())

JVM version: (21, 0, 6)


---

Now that we have imported JPype and started a JVM, we can import and call Java classes.

---

Check the Java classpath and confirm it is configured as intended.

In [9]:
from java.lang import System

print(System.getProperty('java.class.path'))

/Users/dave/research/logmap-matcher-standalone-nov-2025/logmap-matcher-4.0.jar:/Users/dave/research/logmap-matcher-standalone-nov-2025/java-dependencies/sesame-rio-rdfjson-2.7.12.jar:/Users/dave/research/logmap-matcher-standalone-nov-2025/java-dependencies/sesame-rio-n3-2.7.12.jar:/Users/dave/research/logmap-matcher-standalone-nov-2025/java-dependencies/logkit-1.0.1.jar:/Users/dave/research/logmap-matcher-standalone-nov-2025/java-dependencies/sesame-util-2.7.12.jar:/Users/dave/research/logmap-matcher-standalone-nov-2025/java-dependencies/elk-util-collections-0.4.3.jar:/Users/dave/research/logmap-matcher-standalone-nov-2025/java-dependencies/semargl-rdf-0.6.1.jar:/Users/dave/research/logmap-matcher-standalone-nov-2025/java-dependencies/slf4j-log4j12-1.7.31.jar:/Users/dave/research/logmap-matcher-standalone-nov-2025/java-dependencies/xz-1.5.jar:/Users/dave/research/logmap-matcher-standalone-nov-2025/java-dependencies/semargl-core-0.6.1.jar:/Users/dave/research/logmap-matcher-standalone-n

Java imports for basic LogMap usage

In [10]:
from org.semanticweb.owlapi.model import IRI
from org.semanticweb.owlapi.apibinding import OWLManager
from uk.ac.ox.krr.logmap2 import LogMap2_Matcher

Prepare the filepaths of the source and target ontologies the way LogMap expects 

In [11]:
onto1_iri = "file:" + config['alignmentTask']['onto_source']
onto2_iri = "file:" + config['alignmentTask']['onto_target']

Instantiate an OWLAPI ontology manager

In [12]:
onto_manager = OWLManager.createOWLOntologyManager()

Use the ontology manager to load the two ontologies to be aligned

In [13]:
onto1 = onto_manager.loadOntology(IRI.create(onto1_iri))
onto2 = onto_manager.loadOntology(IRI.create(onto2_iri))

Instantiate a LogMap matcher object and, in doing so, consume and align the two ontologies

In [14]:
logmap2_matcher_1 = LogMap2_Matcher(onto1, onto2)

# check type of Java object returned
print(type(logmap2_matcher_1))

<java class 'uk.ac.ox.krr.logmap2.LogMap2_Matcher'>


Retrieve LogMap's initial alignment

In [15]:
mappings_1 = logmap2_matcher_1.getLogmap2_Mappings()

# check type of Java object returned
print(type(mappings_1))

<java class 'java.util.HashSet'>


Check the size of the initial alignment

In [16]:
# call Java method on Java object
print(f"LogMap's initial alignment is empty?: {mappings_1.isEmpty()}")
print()
# call Java method on Java object
print(f'Number of mappings in initial alignment: {mappings_1.size()}')
print()
# call Python method on Java object
print(f'Number of mappings in initial alignment: {len(mappings_1)}')

LogMap's initial alignment is empty?: False

Number of mappings in initial alignment: 1405

Number of mappings in initial alignment: 1405


Convert the initial alignment from a java.util.HashSet object to a Java object array (java.lang.Object[]) so we can access individual mappings within the alignment by indexing

In [17]:
# call Java method on Java object
mappings_1_array = mappings_1.toArray()

# check type of Java object returned
print(type(mappings_1_array))
print(type(mappings_1_array[0]))

<java class 'java.lang.Object[]'>
<java class 'uk.ac.ox.krr.logmap2.mappings.objects.MappingObjectStr'>


Examine the first mapping in LogMap's initial alignment

In [18]:
# use Python-style indexing to access an element of a Java array
mapping = mappings_1_array[0]

#
# display the 5 attributes of a LogMap MappingObjectStr object
#

# an entity from the source ontology
print(mapping.getIRIStrEnt1()) 

# an entity from the target ontology       
print(mapping.getIRIStrEnt2()) 

# the relation: equivalence, subClassOf, superClassOf
# -2 indicates equivalence        
print(mapping.getMappingDirection())  

# LogMap's confidence in the mapping
print(mapping.getConfidence())

# the entity type: classes, properties, individuals
# 0 indicates classes
print(mapping.getTypeOfMapping())

http://mouse.owl#MA_0002668
http://human.owl#NCI_C32827
-2
0.7030000000000001
0


Convert the Java array of MappingObjectStr objects into a Python list of strings

In [19]:
# use a Pythonic approach to manipulating the elements of a
# Java Object array (java.lang.Object[])
mappings_1_list_str = [str(m) for m in mappings_1_array]

# check type of Python objects
print(type(mappings_1_list_str))
print(type(mappings_1_list_str[0]))
print()

# display the first few mappings of the alignment, now represented as Python objects
cnt = 0
for m in mappings_1_list_str:
    cnt += 1
    if cnt > 5:
        break
    print(m)

# NOTE: This simplistic approach to conversion from Java object to Python object
# is insufficient for real use cases.  It is for demonstration only. 
# Only three of the five attributes of the LogMap MappingObjectStr object survive
# this conversion: the IRIs of the two entities, and the MappingDirection.
# The mapping's Type and Confidence are lost. This behaviour is determined by
# the .toString() method of the MappingObjectStr class. Thus, one valuable 
# observation here is that, when using JPype, Python's str() method (for converting 
# things to their string representations) magically invokes the .toString() method
# of Java objects.
#
# The key observation here is that, to ensure we do not lose information, we must
# avoid easy conversion methods and get each attribute of LogMap's MappingObjectStr
# objects individually, as in the preceding code cell.

<class 'list'>
<class 'str'>

<http://mouse.owl#MA_0002668==http://human.owl#NCI_C32827>
<http://mouse.owl#MA_0000378==http://human.owl#NCI_C12416>
<http://mouse.owl#MA_0002492==http://human.owl#NCI_C12331>
<http://mouse.owl#MA_0001746==http://human.owl#NCI_C49323>
<http://mouse.owl#MA_0002418==http://human.owl#NCI_C12754>


Retrieve LogMap's set of 'mappings to ask an Oracle' 

In [20]:
# get LogMap's mappings_to_ask
# (return type: java.util.HashSet of MappingObjectStr objects)
mappings_to_ask = logmap2_matcher_1.getLogmap2_mappings4user()

# apply once again our simplistic Java to Python conversion scheme
# (return type: Python list of strings)
mappings_to_ask_oracle = [str(m) for m in mappings_to_ask.toArray()]

print(f'Number of mappings to ask an Oracle: {len(mappings_to_ask_oracle)}')

# display some of the mappings_to_ask
cnt = 0
for m in mappings_to_ask_oracle:
    cnt += 1
    if cnt > 5:
        break
    print(cnt, m)


Number of mappings to ask an Oracle: 259
1 <http://mouse.owl#MA_0000810==http://human.owl#NCI_C49333>
2 <http://mouse.owl#MA_0002062==http://human.owl#NCI_C33661>
3 <http://mouse.owl#MA_0001130==http://human.owl#NCI_C12350>
4 <http://mouse.owl#MA_0000874==http://human.owl#NCI_C33260>
5 <http://mouse.owl#MA_0002497==http://human.owl#NCI_C33309>


Retrieve LogMap's refined alignment, first passing it the feedback from an Oracle with respect to the mappings_to_ask.  (Here we simulate the predictions of an LLM Oracle by giving LogMap its  original set of mappings_to_ask, pretending they represent Oracle predictions.)

In [21]:
# simulate an Oracle's predictions with LogMap's original mappings_to_ask
mappings_to_ask_oracle_predictions = mappings_to_ask

# have LogMap perform a fresh alignment, but this time whilst taking into 
# consideration the (for now, simulated) Oracle predictions with respect to
# the mappings_to_ask
logmap2_matcher_2 = LogMap2_Matcher(onto1, onto2, mappings_to_ask_oracle_predictions)

# retrieve the refined alignment
mappings_2 = logmap2_matcher_2.getLogmap2_Mappings()

Check the size of the refined alignment

In [22]:
# call Java method on Java object
print(f"LogMap's refined alignment is empty?: {mappings_2.isEmpty()}")
print()
# call Java method on Java object
print(f'Number of mappings in refined alignment: {mappings_2.size()}')
print()
# call Python method on Java object
print(f'Number of mappings in refined alignment: {len(mappings_2)}')

LogMap's refined alignment is empty?: False

Number of mappings in refined alignment: 1409

Number of mappings in refined alignment: 1409
